In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from datetime import datetime
from functools import reduce
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
bangalore_auto = pd.read_csv('indiranagar.csv')
bangalore_auto_hexes = bangalore_auto['pickup_location_hex_8'].unique().tolist()
print(bangalore_auto_hexes)
print(len(bangalore_auto_hexes))

['8861892ec9fffff', '8861892ec7fffff', '8861892ecbfffff', '8861892eddfffff', '8861892ec3fffff', '8861892ed5fffff']
6


In [5]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8, captain_id
from orders.order_logs_snapshot
where yyyymmdd >= '20230417'
and yyyymmdd <= '20230418'
and city_name in ('Bangalore')
and service_obj_service_name = 'Auto'
and pickup_location_hex_8 in ('8861892ec9fffff', '8861892ec7fffff', '8861892ecbfffff', '8861892eddfffff', '8861892ec3fffff', '8861892ed5fffff')
and hour in ('17', '18', '19', '20', '21')
"""


#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [6]:
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id
0,643e86e08a32910b4756e9e8,NaN,Bangalore,8861892ec3fffff,12.979327,77.640686,173240,17,1730,Auto,17:59,2,20230418,customerCancelled,order cancelled before rider accepted,4.0,"[""60effa2e0d1ac71ca89e2fd3"",""5a3e76da730e600b7...",8861892ec3fffff,5c5d54a003a62d32bb1e8d29
1,643e8bed8df51c600d269cbb,NaN,Bangalore,8861892ed5fffff,12.976643,77.635574,175413,17,1745,Auto,17:59,2,20230418,customerCancelled,order cancelled before rider accepted,4.0,"[""60f18e208f3479862d17fcc6"",""610258ab6cd49248c...",8861892ed5fffff,633d22f68592da5bb2fe9ef5
2,643d718de0382b532b688358,NaN,Bangalore,8861892ed5fffff,12.977820,77.634590,214925,21,2145,Auto,21:59,1,20230417,customerCancelled,order cancelled before rider accepted,0.0,"[""5b0a45f0bdfe6955e43b2b8c"",""5cb87a6854bc7263f...",8861892ed5fffff,5cb87a6854bc7263ff4050d0
3,643d70acd4835368cf9cfc45,0.038,Bangalore,8861892ec3fffff,12.978400,77.639053,214540,21,2145,Auto,21:59,1,20230417,dropped,None,1.0,"[""5eb520ec5c1647f36158d15a"",""6309a454732107624...",8861892ec3fffff,5eb520ec5c1647f36158d15a
4,643d6be17624607b38f1cebe,1.674,Bangalore,8861892ec3fffff,12.979215,77.640586,212513,21,2115,Auto,21:00,1,20230417,dropped,None,5.0,"[""60efc71c0d1ac7669b9e0725"",""61b82e034c63be632...",8861892ec3fffff,625faa4e797562a4333d26d4


In [7]:
df[df['order_status'] == 'dropped']

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id
3,643d70acd4835368cf9cfc45,0.038,Bangalore,8861892ec3fffff,12.978400,77.639053,214540,21,2145,Auto,21:59,1,20230417,dropped,None,1.0,"[""5eb520ec5c1647f36158d15a"",""6309a454732107624...",8861892ec3fffff,5eb520ec5c1647f36158d15a
4,643d6be17624607b38f1cebe,1.674,Bangalore,8861892ec3fffff,12.979215,77.640586,212513,21,2115,Auto,21:00,1,20230417,dropped,None,5.0,"[""60efc71c0d1ac7669b9e0725"",""61b82e034c63be632...",8861892ec3fffff,625faa4e797562a4333d26d4
6,643d6aaec314db3050e1f2c5,0.470,Bangalore,8861892ec3fffff,12.978579,77.642849,212006,21,2115,Auto,21:00,1,20230417,dropped,None,2.0,"[""615448638fb56a82f1187a3c"",""6110c7077fa982590...",8861892ec3fffff,618672c87607992b832879eb
9,643d5b8f24bf5168f0ffbbf9,1.923,Bangalore,8861892ecbfffff,12.973970,77.646767,201535,20,2015,Auto,20:00,1,20230417,dropped,None,4.0,"[""605b117d6f09d6585f788bf4"",""61eeb2d37d9d0b14c...",8861892ecbfffff,60e6c63909aa99e889425ef1
17,643d6249e0c4ad606237bbbf,0.321,Bangalore,8861892ec9fffff,12.972536,77.650925,204417,20,2030,Auto,20:59,1,20230417,dropped,None,10.0,"[""62fb335c1549c2ab89220dd2"",""60e44956569b267ea...",8861892ec9fffff,60ec74dfac6ff4faf5627743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,643d416cb53a82389183ccdc,1.250,Bangalore,8861892ec3fffff,12.978543,77.643524,182404,18,1815,Auto,18:00,1,20230417,dropped,None,3.0,"[""62014a9697de560a4b6e919b"",""5d63cf2f357698118...",8861892ec3fffff,619e03163f25c336c1e65a9e
6038,643d457624bf5168f0ff9652,1.060,Bangalore,8861892ec7fffff,12.985682,77.644539,184118,18,1830,Auto,18:59,1,20230417,dropped,None,5.0,"[""62d64887687a18546318652b"",""60e913ccac6ff4c21...",8861892ec7fffff,5a17ff3cd37d16450113ea4c
6048,643d3dbca862bd368cac28be,1.438,Bangalore,8861892ec3fffff,12.978566,77.639343,180820,18,1800,Auto,18:00,1,20230417,dropped,None,2.0,"[""6332f5dce90ec63df578a689"",""60f18e208f3479862...",8861892ec3fffff,6377743a8eedb741c904cd90
6049,643d41a0ae3220266b131ad2,0.811,Bangalore,8861892ecbfffff,12.970280,77.644775,182456,18,1815,Auto,18:00,1,20230417,dropped,None,1.0,"[""61ea758eac28422eb28304d4"",""60f02e6e8f3479842...",8861892ecbfffff,630e2b58c08a963a1a19714b


In [13]:
#Query data from order_logs_snapshot

q = """
select yyyymmdd, epoch, data_orderId, data_propagationBatchId, data_riderId, eventType, hh, data from raw.kafka_dispatch_propagation_immutable
where yyyymmdd >= '20230623'
and hh in ('17', '18', '19', '20', '21')
and data_orderId in ('6495c0a753c99f0f6a376f48')
"""


#Load data into pandas table
df_propagation_copy = pd.read_sql(q, connection)

In [34]:
df_propagation = df_propagation_copy.copy()
df_propagation

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,data
0,20230623,1687535783784,6495c0a753c99f0f6a376f48,985b9bc6-7894-43a0-9550-8b3ea26680f0,648099e413637fceb1e92a2b,rider_acknowledged,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
1,20230623,1687535783784,6495c0a753c99f0f6a376f48,9dc6b69b-455b-48d7-b08a-57eb309cb392,61b83b2e31c1d5c60567b7ff,rider_acknowledged,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
2,20230623,1687535783784,6495c0a753c99f0f6a376f48,56282b04-9cbf-4406-aa01-092c8cb3e3b6,None,propagation_initiated_for_batch,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
3,20230623,1687535783784,6495c0a753c99f0f6a376f48,985b9bc6-7894-43a0-9550-8b3ea26680f0,5d429eab15e40f18eac0f55d,rider_acknowledged,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
4,20230623,1687535783784,6495c0a753c99f0f6a376f48,985b9bc6-7894-43a0-9550-8b3ea26680f0,61de8606d80f2b1a5aa5870a,rider_busy,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
...,...,...,...,...,...,...,...,...
216,20230623,1687535783784,6495c0a753c99f0f6a376f48,985b9bc6-7894-43a0-9550-8b3ea26680f0,5d429eab15e40f18eac0f55d,rider_acknowledged,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
217,20230623,1687535783784,6495c0a753c99f0f6a376f48,985b9bc6-7894-43a0-9550-8b3ea26680f0,648099e413637fceb1e92a2b,rider_acknowledged,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
218,20230623,1687535783784,6495c0a753c99f0f6a376f48,56282b04-9cbf-4406-aa01-092c8cb3e3b6,648099e413637fceb1e92a2b,rider_acknowledged,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
219,20230623,1687535783784,6495c0a753c99f0f6a376f48,56282b04-9cbf-4406-aa01-092c8cb3e3b6,None,propagation_initiated_for_batch,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."


In [35]:
df_filter = df_propagation[df_propagation['eventType'] == 'propagation_initiated_for_batch']
df_filter

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,data
2,20230623,1687535783784,6495c0a753c99f0f6a376f48,56282b04-9cbf-4406-aa01-092c8cb3e3b6,None,propagation_initiated_for_batch,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
52,20230623,1687535783784,6495c0a753c99f0f6a376f48,56282b04-9cbf-4406-aa01-092c8cb3e3b6,None,propagation_initiated_for_batch,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
71,20230623,1687535783784,6495c0a753c99f0f6a376f48,56282b04-9cbf-4406-aa01-092c8cb3e3b6,None,propagation_initiated_for_batch,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
78,20230623,1687535783784,6495c0a753c99f0f6a376f48,9dc6b69b-455b-48d7-b08a-57eb309cb392,None,propagation_initiated_for_batch,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
114,20230623,1687535783784,6495c0a753c99f0f6a376f48,d99679c2-d6ee-4ef2-9a54-609f8d685e8c,None,propagation_initiated_for_batch,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
118,20230623,1687535783784,6495c0a753c99f0f6a376f48,985b9bc6-7894-43a0-9550-8b3ea26680f0,None,propagation_initiated_for_batch,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
126,20230623,1687535783784,6495c0a753c99f0f6a376f48,9dc6b69b-455b-48d7-b08a-57eb309cb392,None,propagation_initiated_for_batch,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
146,20230623,1687535783784,6495c0a753c99f0f6a376f48,e259d92a-0551-4ff6-9549-b7355431512a,None,propagation_initiated_for_batch,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
148,20230623,1687535783784,6495c0a753c99f0f6a376f48,d99679c2-d6ee-4ef2-9a54-609f8d685e8c,None,propagation_initiated_for_batch,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."
159,20230623,1687535783784,6495c0a753c99f0f6a376f48,e259d92a-0551-4ff6-9549-b7355431512a,None,propagation_initiated_for_batch,21,"{""hh"":""21"",""event_quarter_hour"":""2115"",""metada..."


In [36]:
df_filter['data'] = df_filter['data'].apply(lambda x: json.loads(x))
for x in df_filter['data']:
    for captain in x['data']['filteredCaptains']:
        if '636b3ca978125f13dcf4bb4f' == captain['captain']['riderId']:
            print(x['data']['filteredCaptains'])

[{'captain': {'isEligibleForDeprioritization': False, 'isPreferredRoute': False, 'location': {'lat': 17.485893872855755, 'lng': 78.37539933621883}, 'riderId': '636b3ca978125f13dcf4bb4f'}, 'filterType': 'MMO_LIMIT_REACHED'}, {'captain': {'isEligibleForDeprioritization': False, 'isPreferredRoute': False, 'location': {'lat': 17.48980925788865, 'lng': 78.37969623506069}, 'riderId': '5d429eab15e40f18eac0f55d'}, 'filterType': 'MMO_LIMIT_REACHED'}, {'captain': {'isEligibleForDeprioritization': False, 'isPreferredRoute': False, 'location': {'lat': 17.49550630590264, 'lng': 78.38832590728998}, 'riderId': '64551a46b7b8430e762423f6'}, 'filterType': 'MMO_LIMIT_REACHED'}]
[{'captain': {'isEligibleForDeprioritization': False, 'isPreferredRoute': False, 'location': {'lat': 17.485893872855755, 'lng': 78.37539933621883}, 'riderId': '636b3ca978125f13dcf4bb4f'}, 'filterType': 'MMO_LIMIT_REACHED'}, {'captain': {'isEligibleForDeprioritization': False, 'isPreferredRoute': False, 'location': {'lat': 17.48980

In [37]:
df_filter['data'].values[0]

{'hh': '21',
 'event_quarter_hour': '2115',
 'metadata': {'id': '$.eventId',
  'epochFieldsMapping': {'updated_epoch': '$.data.updatedAt',
   'epoch': '$.createdOn'},
  'versionField': '$.__cosmosPlatformMetadata.version'},
 'data': {'maxMMOParallelOrder': 3,
  'orderType': 'auto',
  'orderId': '6495c0a753c99f0f6a376f48',
  'propagationType': 'multicast',
  'riders': ['61de8606d80f2b1a5aa5870a',
   '634c32c18cfdc94e890f7b60',
   '62385748a63f418d05d87dea',
   '648099e413637fceb1e92a2b',
   '5f44bd037da31d089984d207',
   '6272a97a554b1a4f952f6880',
   '5e77656c19f9c87049141bf0'],
  'dropLocation': {'lng': 78.3846664428711, 'lat': 17.470294952392578},
  'pickupLocation': {'lng': 78.38020324707031, 'lat': 17.482942581176758},
  'filteredCaptains': [{'captain': {'isEligibleForDeprioritization': False,
     'isPreferredRoute': False,
     'location': {'lat': 17.485893872855755, 'lng': 78.37539933621883},
     'riderId': '636b3ca978125f13dcf4bb4f'},
    'filterType': 'MMO_LIMIT_REACHED'},
  

In [44]:
df_filter['data'].values[7]['data']['filteredCaptains']

[{'captain': {'isEligibleForDeprioritization': False,
   'isPreferredRoute': False,
   'location': {'lat': 17.4843253288229, 'lng': 78.38976323604584},
   'riderId': '61b83b2e31c1d5c60567b7ff'},
  'filterType': 'MMO_LIMIT_REACHED'}]

In [8]:
df_propagation = df_propagation.sort_values(by = ['data_orderId', 'epoch'])
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh
1692456,20230417,1681733121316,643cef1be0c4ad606236ee32,87c6aaf0-fd49-4e3d-a70c-d36d21cec361,61ac567017dd6a80240a8c6c,rider_acknowledged,17
1814665,20230417,1681733121877,643cef1be0c4ad606236ee32,87c6aaf0-fd49-4e3d-a70c-d36d21cec361,5c1926053f491d0a6e5edeac,rider_acknowledged,17
1794253,20230417,1681733122512,643cef1be0c4ad606236ee32,87c6aaf0-fd49-4e3d-a70c-d36d21cec361,60f7cfdfd417d42a0f7c93d4,rider_acknowledged,17
1797139,20230417,1681733122512,643cef1be0c4ad606236ee32,87c6aaf0-fd49-4e3d-a70c-d36d21cec361,61321d1c422b5a9ba3de1af0,rider_acknowledged,17
1853810,20230417,1681733122523,643cef1be0c4ad606236ee32,87c6aaf0-fd49-4e3d-a70c-d36d21cec361,5c9b93b0b50ab0456be601da,rider_acknowledged,17


In [9]:
df_propagation['counter'] = (df_propagation['data_propagationBatchId'] != df_propagation['data_propagationBatchId'].shift()) | (df_propagation['data_orderId'] != df_propagation['data_orderId'].shift())
df_propagation['counter'] = df_propagation.groupby('data_orderId')['counter'].cumsum()
df_propagation.head()

,yyyymmdd,epoch,data_orderId,data_propagationBatchId,data_riderId,eventType,hh,counter
1692456,20230417,1681733121316,643cef1be0c4ad606236ee32,87c6aaf0-fd49-4e3d-a70c-d36d21cec361,61ac567017dd6a80240a8c6c,rider_acknowledged,17,1
1814665,20230417,1681733121877,643cef1be0c4ad606236ee32,87c6aaf0-fd49-4e3d-a70c-d36d21cec361,5c1926053f491d0a6e5edeac,rider_acknowledged,17,1
1794253,20230417,1681733122512,643cef1be0c4ad606236ee32,87c6aaf0-fd49-4e3d-a70c-d36d21cec361,60f7cfdfd417d42a0f7c93d4,rider_acknowledged,17,1
1797139,20230417,1681733122512,643cef1be0c4ad606236ee32,87c6aaf0-fd49-4e3d-a70c-d36d21cec361,61321d1c422b5a9ba3de1af0,rider_acknowledged,17,1
1853810,20230417,1681733122523,643cef1be0c4ad606236ee32,87c6aaf0-fd49-4e3d-a70c-d36d21cec361,5c9b93b0b50ab0456be601da,rider_acknowledged,17,1


In [10]:
df_propagation_filter = df_propagation[['data_orderId', 'data_riderId', 'counter']]
df_propagation_filter.head()

,data_orderId,data_riderId,counter
1692456,643cef1be0c4ad606236ee32,61ac567017dd6a80240a8c6c,1
1814665,643cef1be0c4ad606236ee32,5c1926053f491d0a6e5edeac,1
1794253,643cef1be0c4ad606236ee32,60f7cfdfd417d42a0f7c93d4,1
1797139,643cef1be0c4ad606236ee32,61321d1c422b5a9ba3de1af0,1
1853810,643cef1be0c4ad606236ee32,5c9b93b0b50ab0456be601da,1


In [11]:
#Merge this on df
df_propagation_filter = df_propagation_filter.rename(columns = {'data_orderId':'order_id', 'data_riderId':'captain_id'})
df = df.merge(df_propagation_filter, on = ['order_id', 'captain_id'], how = 'left')
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,counter
0,643e95e27094c03727fd9e49,NaN,Bangalore,8861892ec3fffff,12.981366,77.641403,183642,18,1830,Auto,18:59,2,20230418,customerCancelled,order cancelled before rider accepted,0.0,"[""60fa1c4cd417d43b2a7dacd1"",""610258ab6cd49248c...",8861892ec3fffff,621321cc2a4d42310ff296d7,2.0
1,643e95e27094c03727fd9e49,NaN,Bangalore,8861892ec3fffff,12.981366,77.641403,183642,18,1830,Auto,18:59,2,20230418,customerCancelled,order cancelled before rider accepted,0.0,"[""60fa1c4cd417d43b2a7dacd1"",""610258ab6cd49248c...",8861892ec3fffff,621321cc2a4d42310ff296d7,3.0
2,643e8fc53dfa3663cedda08b,NaN,Bangalore,8861892eddfffff,12.967491,77.637733,181037,18,1800,Auto,18:00,2,20230418,customerCancelled,order cancelled before rider accepted,5.0,"[""60faa4e222600c43966d608b"",""627a7dadc75b902be...",8861892eddfffff,61a8eced8472c91db85a2d9a,3.0
3,643e926b4120c65287b75242,NaN,Bangalore,8861892ecbfffff,12.970999,77.647522,182155,18,1815,Auto,18:00,2,20230418,customerCancelled,order cancelled before rider accepted,5.0,"[""61eff75075b439863014bcf2"",""5b99b7f498cab328c...",8861892ecbfffff,610258ab6cd49248ca3f8852,4.0
4,643e93897094c03727fd99f9,NaN,Bangalore,8861892ec3fffff,12.978974,77.639369,182641,18,1815,Auto,18:00,2,20230418,customerCancelled,order cancelled before rider accepted,0.0,"[""62e75f76cd87ee7e7b6e4acf"",""636376fcd58af7b1d...",8861892ec3fffff,62e75f76cd87ee7e7b6e4acf,NaN


In [12]:
df['counter'] = df['counter'].fillna(1)
df['counter_group'] = np.where(df['counter'] >= 5, '5 or more batches', df['counter'])
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_id,counter,counter_group
0,643e95e27094c03727fd9e49,NaN,Bangalore,8861892ec3fffff,12.981366,77.641403,183642,18,1830,Auto,18:59,2,20230418,customerCancelled,order cancelled before rider accepted,0.0,"[""60fa1c4cd417d43b2a7dacd1"",""610258ab6cd49248c...",8861892ec3fffff,621321cc2a4d42310ff296d7,2.0,2.0
1,643e95e27094c03727fd9e49,NaN,Bangalore,8861892ec3fffff,12.981366,77.641403,183642,18,1830,Auto,18:59,2,20230418,customerCancelled,order cancelled before rider accepted,0.0,"[""60fa1c4cd417d43b2a7dacd1"",""610258ab6cd49248c...",8861892ec3fffff,621321cc2a4d42310ff296d7,3.0,3.0
2,643e8fc53dfa3663cedda08b,NaN,Bangalore,8861892eddfffff,12.967491,77.637733,181037,18,1800,Auto,18:00,2,20230418,customerCancelled,order cancelled before rider accepted,5.0,"[""60faa4e222600c43966d608b"",""627a7dadc75b902be...",8861892eddfffff,61a8eced8472c91db85a2d9a,3.0,3.0
3,643e926b4120c65287b75242,NaN,Bangalore,8861892ecbfffff,12.970999,77.647522,182155,18,1815,Auto,18:00,2,20230418,customerCancelled,order cancelled before rider accepted,5.0,"[""61eff75075b439863014bcf2"",""5b99b7f498cab328c...",8861892ecbfffff,610258ab6cd49248ca3f8852,4.0,4.0
4,643e93897094c03727fd99f9,NaN,Bangalore,8861892ec3fffff,12.978974,77.639369,182641,18,1815,Auto,18:00,2,20230418,customerCancelled,order cancelled before rider accepted,0.0,"[""62e75f76cd87ee7e7b6e4acf"",""636376fcd58af7b1d...",8861892ec3fffff,62e75f76cd87ee7e7b6e4acf,1.0,1.0


In [13]:
#Filter only on the hexes needed
df_filter_level1 = df
df_filter_level1['count'] = 1
df_filter_level1['date'] = df_filter_level1['yyyymmdd'].astype(str).apply(lambda x: x[6:])

In [14]:
#Create a new column for cancel_reason to identify OCARA
df_filter_level1['new_cancel_reason'] = np.where(df_filter_level1['order_status'] == 'customerCancelled',
 np.where(df_filter_level1['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_filter_level1['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')

In [15]:
#Check if the order is expiry_mapped or expiry_unmapped
df_filter_level1['expiry_type'] = np.where(df_filter_level1['order_status'] == 'expired',
 np.where(df_filter_level1['map_riders'] == '[]', 'expiry_unmapped', 'expiry_mapped'), 'no_expiry')
df_filter_level1['final_order_status'] = np.where(df_filter_level1['order_status'] == 'customerCancelled', df_filter_level1['new_cancel_reason'],
np.where(df_filter_level1['order_status'] == 'expired', df_filter_level1['expiry_type'], df_filter_level1['order_status']))

In [16]:
#Metrics_pivot
metrics_pivot = pd.pivot_table(df_filter_level1, index = ['yyyymmdd', 'counter_group'], columns = 'final_order_status',
 values = 'count', aggfunc = 'sum', fill_value=0).reset_index().rename_axis(None, axis=1)
metrics_pivot['total'] = metrics_pivot[['COBRA', 'COBRM', 'OCARA', 'aborted', 'dropped', 'expiry_mapped', 'expiry_unmapped']].sum(axis = 1)
metrics_pivot['OCARA%'] = round(metrics_pivot['OCARA']/metrics_pivot['total'], 4)
metrics_pivot['COBRA%'] = round(metrics_pivot['COBRA']/metrics_pivot['total'], 4)
metrics_pivot['COBRM%'] = round(metrics_pivot['COBRM']/metrics_pivot['total'], 4)
metrics_pivot['G2N'] = round(metrics_pivot['dropped']/metrics_pivot['total'], 4)
metrics_pivot['expiry_mapped%'] = round(metrics_pivot['expiry_mapped']/metrics_pivot['total'], 4)
if 'expiry_unmapped' in metrics_pivot.columns:
    metrics_pivot['Stockout%'] = round(metrics_pivot['expiry_unmapped']/metrics_pivot['total'], 4)
else:
    metrics_pivot['Stockout%'] = 0
metrics_df = metrics_pivot[['yyyymmdd', 'counter_group', 'OCARA%', 'COBRA%', 'COBRM%', 'G2N', 'expiry_mapped%', 'Stockout%', 'total']]
metrics_df

KeyError: "['COBRM'] not in index"

In [64]:
#Get the average ETA and FM
df_dropped = df_filter_level1[df_filter_level1['final_order_status'] == 'dropped']
fm = df_dropped.groupby(['yyyymmdd', 'counter_group'])['accept_to_pickup_distance'].median().reset_index()
fm = fm.rename(columns = {'accept_to_pickup_distance':'FM'})
eta = df_dropped.groupby(['yyyymmdd', 'counter_group'])['eta'].median().reset_index()
metrics_df = metrics_df.merge(fm, on = ['yyyymmdd', 'counter_group'], how = 'left')
metrics_df = metrics_df.merge(eta, on = ['yyyymmdd', 'counter_group'], how = 'left')
metrics_df.head()

,yyyymmdd,counter_group,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta
0,20230417,1.0,0.0612,0.4615,0.0001,0.2649,0.2114,0.0001,10454,0.6850,3.0
1,20230417,2.0,0.0738,0.5051,0.0000,0.2289,0.1916,0.0000,4500,0.8165,3.0
2,20230417,3.0,0.0566,0.4134,0.0000,0.1649,0.3643,0.0000,4683,0.7500,3.0
3,20230417,4.0,0.0401,0.2120,0.0000,0.0772,0.6704,0.0000,5480,0.7570,3.0
4,20230417,5 or more batches,0.0234,0.0816,0.0000,0.0481,0.8467,0.0000,7063,0.7700,3.0


In [65]:
metrics_df

,yyyymmdd,counter_group,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta
0,20230417,1.0,0.0612,0.4615,0.0001,0.2649,0.2114,0.0001,10454,0.6850,3.0
1,20230417,2.0,0.0738,0.5051,0.0000,0.2289,0.1916,0.0000,4500,0.8165,3.0
2,20230417,3.0,0.0566,0.4134,0.0000,0.1649,0.3643,0.0000,4683,0.7500,3.0
3,20230417,4.0,0.0401,0.2120,0.0000,0.0772,0.6704,0.0000,5480,0.7570,3.0
4,20230417,5 or more batches,0.0234,0.0816,0.0000,0.0481,0.8467,0.0000,7063,0.7700,3.0
5,20230418,1.0,0.0620,0.4518,0.0004,0.2787,0.2062,0.0001,10301,0.6870,3.0
6,20230418,2.0,0.0699,0.5101,0.0000,0.2138,0.2050,0.0000,4419,0.7970,3.0
7,20230418,3.0,0.0625,0.3872,0.0000,0.1792,0.3707,0.0000,4783,0.7330,3.0
8,20230418,4.0,0.0376,0.2154,0.0000,0.0821,0.6644,0.0000,5348,0.7980,3.0
9,20230418,5 or more batches,0.0223,0.0902,0.0000,0.0465,0.8409,0.0000,7362,0.7890,3.0


In [57]:
metrics_df

,yyyymmdd,counter_group,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM_x,eta_x,FM_y,eta_y
0,20230417,1.0,0.0612,0.4615,0.0001,0.2649,0.2114,0.0001,10454,3.404303,3.072228,0.6850,3.0
1,20230417,2.0,0.0738,0.5051,0.0000,0.2289,0.1916,0.0000,4500,16.989028,3.336893,0.8165,3.0
2,20230417,3 or more batches,0.0377,0.2133,0.0000,0.0891,0.6595,0.0000,17226,3.227829,3.340065,0.7570,3.0
3,20230418,1.0,0.0620,0.4518,0.0004,0.2787,0.2062,0.0001,10301,3.313941,3.058516,0.6870,3.0
4,20230418,2.0,0.0699,0.5101,0.0000,0.2138,0.2050,0.0000,4419,16.525852,3.299471,0.7970,3.0
5,20230418,3 or more batches,0.0380,0.2097,0.0000,0.0936,0.6584,0.0000,17493,7.587716,3.266178,0.7560,3.0
